In [ ]:

import openai
import json
from pykrx import stock
from pykrx import bond

TICKER_SAMSUNG = "005930"

SYSTEM_PROMPT = """
            너는 삼성전자 주식 정보를 알려주는 봇이야. 
            아래와 같은 질문이 온 경우에만 대답해야해
            1. 삼성 또는 삼성전자의 주식 정보
            2. 날짜를 말해야 함
            
            CASE1
            질문이 올바르지 않은 경우 아래와 같은 대답을 출력해줘
            1. 삼성 또는 삼성전자가 아닌 다른 회사의 주식 정보를 물어본 경우
              -> 저는 삼성전자의 주식 정보만 대답할 수 있습니다.
            2. 날짜를 말하지 않은 경우
              -> 날짜의 범위를 지정해서 다시 질문해 주세요.
            3. 그 외 엉뚱한 질문인 경우
              -> 무슨 말인지 모르겠어요, 삼성전자의 주식 정보를 날짜 범위를 지정해 질문해 주세요.

            CASE2
            질문이 올바른 경우 아래와 같은 JSON 형식으로 응답해줘
            {
                start_dt: 사용자가 말한 시작일
                end_dt: 사용자가 말한 종료일
            } 
"""

def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except (TypeError, ValueError):
        return False

def get_market_fundamental(start_dt, end_dt, ticket):
    df = stock.get_market_fundamental(start_dt, end_dt, ticket)
    return df.head(365)

def get_market_ohlcv(start_dt, end_dt, ticket):
    df = stock.get_market_ohlcv(start_dt, end_dt, ticket)
    return df.head(365)

def chat_create(question, prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user", 
                "content": question
            }],
        )
    return response.choices[0].message.content

def ask(question, prompt, cache):
    if(prompt != ""):
       cache = chat_create(question, prompt)
       return ask(question, "", cache)
    else:
        if not is_valid_json(cache):
            return cache

        ref = json.loads(cache)
        start_dt = ref["start_dt"]
        end_dt = ref["end_dt"]
        fundamental = get_market_fundamental(start_dt, end_dt, TICKER_SAMSUNG)
        ohlcv = get_market_ohlcv(start_dt, end_dt, TICKER_SAMSUNG)

        fundamental_dict = fundamental.to_dict(orient="records")
        ohlcv_dict = ohlcv.to_dict(orient="records")

        fundamental_str = json.dumps(fundamental_dict, ensure_ascii=False)
        ohlcv_str = json.dumps(ohlcv_dict, ensure_ascii=False)

        question = (
            "질문과 데이터를 바탕으로 분석해서 결과를 알려줘"
            + "질문 : " + question
            + ", 데이터1 : " + fundamental_str
            + ", 데이터2 : " + ohlcv_str
        )
        return chat_create(question, "")

def chat():
    print("삼성전자의 주식 정보에 대해 날짜 범위와 함께 질문해 주세요.")
    while True:
        user_input = input("User: ")
        print(f"USER: {user_input}")

        if user_input.lower() == 'exit':
            print("Exiting...")
            break
        response = ask(user_input, SYSTEM_PROMPT, "")
        print(f"GPT: {response}")

if __name__ == "__main__":
    chat()


               Open     High      Low    Close     Volume  Adj Close
2019-12-31  5.77250  5.89200  5.75325  5.88250  231004000   5.857942
2020-01-02  5.96875  5.99775  5.91800  5.99775  237536000   5.972711
2020-01-03  5.87750  5.94575  5.85250  5.90175  205384000   5.877110
2020-01-06  5.80800  5.93175  5.78175  5.92650  262636000   5.901758
2020-01-07  5.95500  6.04425  5.90975  5.99825  314856000   5.973208
2020-01-08  5.99400  6.05100  5.95375  6.00950  277108000   5.984412
2020-01-09  6.09625  6.14825  6.02150  6.07550  255112000   6.050136
2020-01-10  6.18325  6.21375  6.09375  6.10800  316296000   6.082500
2020-01-13  6.19150  6.32475  6.16875  6.29950  319840000   6.273201
2020-01-14  6.25650  6.28200  6.16875  6.18200  359088000   6.156192
2020-01-15  6.19475  6.21725  6.11325  6.13925  263104000   6.113620
2020-01-16  6.19300  6.22925  6.16950  6.22325  284192000   6.197269
2020-01-17  6.24250  6.25000  6.18425  6.23200  253792000   6.205983
2020-01-21  6.19525  6.23225  6.16